In [34]:
import transformers as t
import datasets
from datasets import load_dataset

import torch
import torch_geometric

import numpy as np
import pandas as pd

import spacy

import os
import json

from torch_geometric.data import Data, Dataset
from huggingface_hub import hf_hub_download

from huggingface_hub import hf_hub_url
from huggingface_hub.file_download import cached_download

In [31]:
def graphs_from_json(json_file):
    with open(json_file, "r") as f:
        graphs = json.load(f)

    dataset = []
    for graph in graphs:
        x = torch.tensor(graph['x'], dtype=torch.float).view(-1, 1)
        edge_index = torch.tensor(graph['edge_index'], dtype=torch.long)
        y = torch.tensor(graph['y'], dtype=torch.long)
        data = Data(x=x, edge_index=edge_index, y=y)
        dataset.append(data)

    return dataset

In [35]:
repo_name = "FranklinWillemen/mapa_plus_graph"

# Download the JSON files
#hf_hub_download(repo_id=repo_name, repo_type="dataset", filename="graphs_train.json")
#hf_hub_download(repo_id=repo_name, repo_type="dataset", filename="graphs_val.json")
#hf_hub_download(repo_id=repo_name, repo_type="dataset", filename="graphs_test.json")

train_url = hf_hub_url(repo_name, "graphs_train.json", repo_type="dataset")
val_url = hf_hub_url(repo_name, "graphs_val.json", repo_type="dataset")
test_url = hf_hub_url(repo_name, "graphs_test.json", repo_type="dataset")

train_file = cached_download(train_url)
val_file = cached_download(val_url)
test_file = cached_download(test_url)

train_dataset = graphs_from_json(train_file)
val_dataset = graphs_from_json(val_file)
test_dataset = graphs_from_json(test_file)